# Use k means cluster on data to get classified labels

## Importing and Read & Preprocess data

In [1]:
import global_resources as gr
import numpy as np
import torch
import kmc_torch.kmc as kmc
import os
import pandas as pd

'CAPM' directory is ready at: D:\Important Files\Repositories\Quantitative-Investment-Algorithms\CAPMlib
'K means cluster' directory is ready at: D:\Important Files\Repositories\Quantitative-Investment-Algorithms\kmc_torch
'Data' directory is ready at: D:\Important Files\Repositories\Quantitative-Investment-Algorithms\Data
'SVM' directory is ready at: D:\Important Files\Repositories\Quantitative-Investment-Algorithms\SVM
'Output' directory is ready at: D:\Important Files\Repositories\Quantitative-Investment-Algorithms\Output


In [2]:
# data_dir = r"D:\Important Files\Repositories\Quantitative-Investment-Algorithms\Data\Cluster DATA\processed_data_final.csv"
data_dir = os.path.join(gr.default_dir, r'Data\Cluster DATA\processed_data_final.csv')
df = gr.read_and_return_pd_df(data_dir)
to_drop = ['证券简称', '行业代码1']
df.drop(to_drop, axis = 1, inplace = True)
df['证券代码'] = df['证券代码'].astype(str).str.zfill(6)
Stock_ID_df = df['证券代码']
df.drop(['证券代码'], axis = 1, inplace = True)
display(Stock_ID_df)
display(df)

Reading files from: 'D:\Important Files\Repositories\Quantitative-Investment-Algorithms\Data\Cluster DATA\processed_data_final.csv'


0       000001
1       000002
2       000004
3       000006
4       000007
         ...  
5316    920002
5317    920008
5318    920016
5319    920099
5320    920118
Name: 证券代码, Length: 5321, dtype: object

,营业总收入,现金资产比率,应收类资产比率,固定资产比率,无形资产比率,金融负债比率,少数股东权益占比,市盈率（PE）1,资产负债率,市值A,托宾Q值A,账面市值比A,资产报酬率A,净资产收益率（ROE）A
0,1.294663,-0.777553,-1.270996,-1.297656,-0.692415,-1.477725,-0.185208,-0.330143,2.323068,4.379345,-0.733083,1.421790,-0.375351,0.132565
1,3.324778,-0.706152,-1.208915,-1.205924,-0.608636,-0.188667,1.208350,-0.326349,1.446345,0.858528,-0.829444,1.958758,-0.737222,-0.103471
2,-0.151864,0.133231,2.573513,-1.277036,-0.136708,-1.085169,-0.257693,-0.370844,0.840408,-0.068671,3.617179,-2.012428,-2.865983,-0.471754
3,-0.076711,-0.165453,-1.251412,-1.287878,-0.710005,0.400996,0.012098,-0.370844,1.215474,-0.053009,-0.664129,1.108182,-1.008012,-0.196261
4,-0.150626,1.671150,-0.842090,-0.384225,-0.707361,0.594283,0.100501,0.064576,0.780991,-0.068667,4.447951,-2.103008,-0.356346,0.001572
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5316,-0.148916,0.255536,0.050418,0.063653,-0.494292,-1.182638,-0.185208,-0.140737,-1.544378,-0.069825,-0.522023,0.592871,0.969871,0.106275
5317,-0.149857,0.215799,3.219931,-0.949145,-0.670456,0.700225,-0.185208,-0.148138,-0.198913,-0.069806,0.143840,-0.678933,1.255378,0.184321
5318,-0.150593,0.936079,-0.717003,0.952690,0.067164,1.518683,-0.185208,-0.019689,0.223056,-0.069697,-0.208876,-0.160995,0.756657,0.153632
5319,-0.148657,1.784303,-0.655317,-1.041496,0.039194,-0.601230,-0.185208,-0.230032,0.294809,-0.069054,-0.508312,0.550499,0.700999,0.165490


## Move data to gpu

In [3]:
device = gr.set_device()
print(f"Current device: {device.capitalize()}.")
X = np.array(df, dtype = np.float64)
X_gpu = torch.tensor(X, device = device, dtype = torch.float64)

Current device: Cuda.


## K is set as 10, which is based on the elbow method, and run wcss for single k function to get the labels

In [4]:
X, y, centroids, var = kmc.WCSS_for_single_k(X = X_gpu, k = 10, n_restarts = 50)
print(X.shape)
print(y.shape)
print(centroids.shape)
print(var)

The seed starts at 2634854452
Clustering with: k = 10.
Initiating centroids with k being 10...
Initiating centroids with k being 10...
Initiating centroids with k being 10...
Initiating centroids with k being 10...
Initiating centroids with k being 10...
Initiating centroids with k being 10...
Initiating centroids with k being 10...
Initiating centroids with k being 10...
Initiating centroids with k being 10...
Initiating centroids with k being 10...
Initiating centroids with k being 10...
Initiating centroids with k being 10...
Initiating centroids with k being 10...
Initiating centroids with k being 10...
Initiating centroids with k being 10...
Initiating centroids with k being 10...
Initiating centroids with k being 10...
Initiating centroids with k being 10...
Initiating centroids with k being 10...
Initiating centroids with k being 10...
Initiating centroids with k being 10...
Initiating centroids with k being 10...
Initiating centroids with k being 10...
Initiating centroids with

In [5]:
k_results = y.detach().cpu()
k_results = k_results.numpy()
k_results = pd.DataFrame(k_results)
k_results = pd.concat([Stock_ID_df, k_results], axis = 1)
cols = ['ID', 'K_Results']
k_results.columns = cols
display(k_results)

,ID,K_Results
0,000001,0
1,000002,0
2,000004,1
3,000006,0
4,000007,1
...,...,...
5316,920002,2
5317,920008,7
5318,920016,4
5319,920099,3


# SVC/OVO

In [6]:
import SVM.SVC as svc

Current training device: Cuda.


## Set parameters

In [7]:
lr_cpu = 0.01
lr = torch.tensor(lr_cpu, device = X.device, dtype = X.dtype)

In [8]:
%time
wb_dic = svc.ovo_train(
    X,
    y,
    num_epochs = int(1e4),
    start_learning_rate = lr,
    l2_penalty = True,
    print_every = int(1e3),
    delta_loss_breaker = 1e-7,
    patience = int(10),
    relative = True,
    relative_breaker = 1e-6
)

CPU times: total: 0 ns
Wall time: 0 ns
-------------------------------------------------------------------------------------------------------
Training on label a: 0 and label b: 1
Creating random weights and bias with dtype: torch.float64
Training with loss function: hinge loss with l2 penalty on weights.
Training with relative breaker.
Epoch 0 | Loss: 0.8209174259423025 | Relative Ratio: None
Epoch 1 | Loss: 0.8008959452689957 | Relative Ratio: 1.0249988538356032
Epoch 571 | Loss: 0.0030558518300606973 | Relative Ratio: 1.000000439061118
Exited with relative_ratio consecutively being smaller than 1e-06 from epoch 561 to epoch 571.
-------------------------------------------------------------------------------------------------------
Training on label a: 0 and label b: 2
Creating random weights and bias with dtype: torch.float64
Training with loss function: hinge loss with l2 penalty on weights.
Training with relative breaker.
Epoch 0 | Loss: 1.3097340786718517 | Relative Ratio: None


In [9]:
votes = svc.ovo_predict(wb_dic, X, y.dtype)

In [10]:
svc.ovo_score(X, y, wb_dic)

Accuracy: 5253/5321 = 98.72%


0.987220447284345

In [11]:
votes_df = gr.detach_to_pd(votes)
display(votes_df)

,0
0,0
1,0
2,1
3,0
4,1
...,...
5316,2
5317,7
5318,4
5319,3


In [12]:
df = pd.concat([Stock_ID_df, votes_df], axis = 1)
COLS = ['ID', 'SVC_votes']
df.columns = COLS
display(df)

,ID,SVC_votes
0,000001,0
1,000002,0
2,000004,1
3,000006,0
4,000007,1
...,...,...
5316,920002,2
5317,920008,7
5318,920016,4
5319,920099,3


# Classified Data Analysis

In [13]:
compare_df = pd.merge(df, k_results)
display(compare_df)

,ID,SVC_votes,K_Results
0,000001,0,0
1,000002,0,0
2,000004,1,1
3,000006,0,0
4,000007,1,1
...,...,...,...
5316,920002,2,2
5317,920008,7,7
5318,920016,4,4
5319,920099,3,3


In [14]:
different = compare_df[(compare_df['SVC_votes'] != compare_df['K_Results'])]

In [1]:
different

NameError: name 'different' is not defined

In [15]:
display(different)
print(different['K_Results'].unique())
print(different['SVC_votes'].unique())

,ID,SVC_votes,K_Results
140,000564,4,0
168,000607,0,2
219,000695,6,4
238,000719,0,2
394,000967,6,0
...,...,...,...
4908,688516,2,7
4915,688525,4,1
5003,688646,0,2
5008,688655,4,2


[0 2 4 6 7 1 5 3]
[4 0 6 7 2 1 3 9 5]


In [16]:
data_df = gr.read_and_return_pd_df(r"Data\Cluster Data\processed_data_final.csv")
data_df['证券代码'] = data_df['证券代码'].astype(str).str.zfill(6)
merged = pd.merge(data_df, compare_df, left_on = '证券代码', right_on = 'ID')
merged.drop('ID', axis = 1, inplace = True)
display(merged)

Reading files from: 'Data\Cluster Data\processed_data_final.csv'


,证券代码,证券简称,营业总收入,行业代码1,现金资产比率,应收类资产比率,固定资产比率,无形资产比率,金融负债比率,少数股东权益占比,市盈率（PE）1,资产负债率,市值A,托宾Q值A,账面市值比A,资产报酬率A,净资产收益率（ROE）A,SVC_votes,K_Results
0,000001,平安银行,1.294663,J66,-0.777553,-1.270996,-1.297656,-0.692415,-1.477725,-0.185208,-0.330143,2.323068,4.379345,-0.733083,1.421790,-0.375351,0.132565,0,0
1,000002,万科A,3.324778,K70,-0.706152,-1.208915,-1.205924,-0.608636,-0.188667,1.208350,-0.326349,1.446345,0.858528,-0.829444,1.958758,-0.737222,-0.103471,0,0
2,000004,国华网安,-0.151864,I65,0.133231,2.573513,-1.277036,-0.136708,-1.085169,-0.257693,-0.370844,0.840408,-0.068671,3.617179,-2.012428,-2.865983,-0.471754,1,1
3,000006,深振业A,-0.076711,K70,-0.165453,-1.251412,-1.287878,-0.710005,0.400996,0.012098,-0.370844,1.215474,-0.053009,-0.664129,1.108182,-1.008012,-0.196261,0,0
4,000007,全新好,-0.150626,K70,1.671150,-0.842090,-0.384225,-0.707361,0.594283,0.100501,0.064576,0.780991,-0.068667,4.447951,-2.103008,-0.356346,0.001572,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5316,920002,万达轴承,-0.148916,C34,0.255536,0.050418,0.063653,-0.494292,-1.182638,-0.185208,-0.140737,-1.544378,-0.069825,-0.522023,0.592871,0.969871,0.106275,2,2
5317,920008,成电光信,-0.149857,C39,0.215799,3.219931,-0.949145,-0.670456,0.700225,-0.185208,-0.148138,-0.198913,-0.069806,0.143840,-0.678933,1.255378,0.184321,7,7
5318,920016,中草香料,-0.150593,C26,0.936079,-0.717003,0.952690,0.067164,1.518683,-0.185208,-0.019689,0.223056,-0.069697,-0.208876,-0.160995,0.756657,0.153632,4,4
5319,920099,瑞华技术,-0.148657,M74,1.784303,-0.655317,-1.041496,0.039194,-0.601230,-0.185208,-0.230032,0.294809,-0.069054,-0.508312,0.550499,0.700999,0.165490,3,3


In [17]:
merged.rename(columns={'证券代码': 'ID'}, inplace=True)

## 丢掉k和svc判断不一样的股票，然后将两个结果合并

In [18]:
merged.drop(index = different.index, inplace = True)

In [19]:
display(merged)

,ID,证券简称,营业总收入,行业代码1,现金资产比率,应收类资产比率,固定资产比率,无形资产比率,金融负债比率,少数股东权益占比,市盈率（PE）1,资产负债率,市值A,托宾Q值A,账面市值比A,资产报酬率A,净资产收益率（ROE）A,SVC_votes,K_Results
0,000001,平安银行,1.294663,J66,-0.777553,-1.270996,-1.297656,-0.692415,-1.477725,-0.185208,-0.330143,2.323068,4.379345,-0.733083,1.421790,-0.375351,0.132565,0,0
1,000002,万科A,3.324778,K70,-0.706152,-1.208915,-1.205924,-0.608636,-0.188667,1.208350,-0.326349,1.446345,0.858528,-0.829444,1.958758,-0.737222,-0.103471,0,0
2,000004,国华网安,-0.151864,I65,0.133231,2.573513,-1.277036,-0.136708,-1.085169,-0.257693,-0.370844,0.840408,-0.068671,3.617179,-2.012428,-2.865983,-0.471754,1,1
3,000006,深振业A,-0.076711,K70,-0.165453,-1.251412,-1.287878,-0.710005,0.400996,0.012098,-0.370844,1.215474,-0.053009,-0.664129,1.108182,-1.008012,-0.196261,0,0
4,000007,全新好,-0.150626,K70,1.671150,-0.842090,-0.384225,-0.707361,0.594283,0.100501,0.064576,0.780991,-0.068667,4.447951,-2.103008,-0.356346,0.001572,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5316,920002,万达轴承,-0.148916,C34,0.255536,0.050418,0.063653,-0.494292,-1.182638,-0.185208,-0.140737,-1.544378,-0.069825,-0.522023,0.592871,0.969871,0.106275,2,2
5317,920008,成电光信,-0.149857,C39,0.215799,3.219931,-0.949145,-0.670456,0.700225,-0.185208,-0.148138,-0.198913,-0.069806,0.143840,-0.678933,1.255378,0.184321,7,7
5318,920016,中草香料,-0.150593,C26,0.936079,-0.717003,0.952690,0.067164,1.518683,-0.185208,-0.019689,0.223056,-0.069697,-0.208876,-0.160995,0.756657,0.153632,4,4
5319,920099,瑞华技术,-0.148657,M74,1.784303,-0.655317,-1.041496,0.039194,-0.601230,-0.185208,-0.230032,0.294809,-0.069054,-0.508312,0.550499,0.700999,0.165490,3,3


## 将列名改为Labels

In [20]:
merged.rename(columns={'SVC_votes': 'Labels'}, inplace=True)
merged.drop('K_Results', axis = 1, inplace = True)
display(merged)

,ID,证券简称,营业总收入,行业代码1,现金资产比率,应收类资产比率,固定资产比率,无形资产比率,金融负债比率,少数股东权益占比,市盈率（PE）1,资产负债率,市值A,托宾Q值A,账面市值比A,资产报酬率A,净资产收益率（ROE）A,Labels
0,000001,平安银行,1.294663,J66,-0.777553,-1.270996,-1.297656,-0.692415,-1.477725,-0.185208,-0.330143,2.323068,4.379345,-0.733083,1.421790,-0.375351,0.132565,0
1,000002,万科A,3.324778,K70,-0.706152,-1.208915,-1.205924,-0.608636,-0.188667,1.208350,-0.326349,1.446345,0.858528,-0.829444,1.958758,-0.737222,-0.103471,0
2,000004,国华网安,-0.151864,I65,0.133231,2.573513,-1.277036,-0.136708,-1.085169,-0.257693,-0.370844,0.840408,-0.068671,3.617179,-2.012428,-2.865983,-0.471754,1
3,000006,深振业A,-0.076711,K70,-0.165453,-1.251412,-1.287878,-0.710005,0.400996,0.012098,-0.370844,1.215474,-0.053009,-0.664129,1.108182,-1.008012,-0.196261,0
4,000007,全新好,-0.150626,K70,1.671150,-0.842090,-0.384225,-0.707361,0.594283,0.100501,0.064576,0.780991,-0.068667,4.447951,-2.103008,-0.356346,0.001572,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5316,920002,万达轴承,-0.148916,C34,0.255536,0.050418,0.063653,-0.494292,-1.182638,-0.185208,-0.140737,-1.544378,-0.069825,-0.522023,0.592871,0.969871,0.106275,2
5317,920008,成电光信,-0.149857,C39,0.215799,3.219931,-0.949145,-0.670456,0.700225,-0.185208,-0.148138,-0.198913,-0.069806,0.143840,-0.678933,1.255378,0.184321,7
5318,920016,中草香料,-0.150593,C26,0.936079,-0.717003,0.952690,0.067164,1.518683,-0.185208,-0.019689,0.223056,-0.069697,-0.208876,-0.160995,0.756657,0.153632,4
5319,920099,瑞华技术,-0.148657,M74,1.784303,-0.655317,-1.041496,0.039194,-0.601230,-0.185208,-0.230032,0.294809,-0.069054,-0.508312,0.550499,0.700999,0.165490,3


## 将ID，证卷名字，label保存下来

In [ ]:
informations = merged[['ID', '证券简称', 'Labels']]
informations.to_csv(os.path.join(gr.global_paths['Output'], 'k=10_processed_data.csv'), index = False, encoding='utf-8')

# CAPM on Each Cluster

In [22]:
import CAPMlib.CAPM as capm

Reading files from: 'D:\Important Files\Repositories\Quantitative-Investment-Algorithms\Data\market_index.csv'
2024-01-02
2025-01-02
Reading files from: 'D:\Important Files\Repositories\Quantitative-Investment-Algorithms\Data\Stock Data\000001_daily_hfq.csv'
Reading files from: 'D:\Important Files\Repositories\Quantitative-Investment-Algorithms\Data\Stock Data\000002_daily_hfq.csv'
Reading files from: 'D:\Important Files\Repositories\Quantitative-Investment-Algorithms\Data\Stock Data\000004_daily_hfq.csv'
Reading files from: 'D:\Important Files\Repositories\Quantitative-Investment-Algorithms\Data\Stock Data\000006_daily_hfq.csv'
Reading files from: 'D:\Important Files\Repositories\Quantitative-Investment-Algorithms\Data\Stock Data\000007_daily_hfq.csv'
Reading files from: 'D:\Important Files\Repositories\Quantitative-Investment-Algorithms\Data\Stock Data\000009_daily_hfq.csv'
Reading files from: 'D:\Important Files\Repositories\Quantitative-Investment-Algorithms\Data\Stock Data\000014_

## 读取labels

In [28]:
labels = gr.read_and_return_pd_df(os.path.join(gr.global_paths['Output'], 'k=10_processed_data.csv'))
labels['ID'] = labels['ID'].astype(str).str.zfill(6)

Reading files from: 'D:\Important Files\Repositories\Quantitative-Investment-Algorithms\Output\k=10_processed_data.csv'


## 按照label分类

In [29]:
label_dfs = capm.label_dfs(labels, label_column = 'Labels')

## 得到所有stock的时间序列数据

In [30]:
time_series_dfs = capm.dfs

## 合并成三维网状字典

In [31]:
dfs = capm.create_nested_dfs(label_dfs = label_dfs, time_series_dfs = time_series_dfs, id_column = 'ID')

  - Label: 0, Stock ID: 000011
  - Label: 0, Stock ID: 000016
  - Label: 0, Stock ID: 000030
  - Label: 0, Stock ID: 000034
  - Label: 0, Stock ID: 000036
  - Label: 0, Stock ID: 000040
  - Label: 0, Stock ID: 000042
  - Label: 0, Stock ID: 000055
  - Label: 0, Stock ID: 000056
  - Label: 0, Stock ID: 000065
  - ... and 170 more
Note: 103 stock IDs in time_series_dfs are not associated with any label
First 10 unused IDs: ['300448', '600189', '002429', '300268', '688646', '002354', '000692', '603290', '300125', '002346']


注释：这些跳过的主要为cluster数据（Data\Cluster DATA\processed_data_final.csv）中本身就没有的股票；一部分为在选定时间序列中没有完整数据的股票；还有一部分为svc和kmc结果不一样的股票

## 得到每簇每天平均涨跌幅dfs

In [32]:
result = {}
for label, stock_dfs in dfs.items():
    # Check if the cluster has any stocks
    if not stock_dfs:
        print(f"No stocks found in cluster {label}, skipping...")
        continue
        
    # Initialize a DataFrame with just the date column from the first stock
    first_stock_id = list(stock_dfs.keys())[0]
    time_series_df = pd.DataFrame({'日期': stock_dfs[first_stock_id]['日期']})
    
    # Collect percentage changes from all stocks in this cluster
    all_pct_changes = []
    for stock_id, df in stock_dfs.items():
        # Ensure alignment with the date column
        stock_pct = df.set_index('日期')['涨跌幅']
        all_pct_changes.append(stock_pct)
    
    # Combine all percentage changes and calculate the mean
    combined_pct = pd.concat(all_pct_changes, axis=1)
    avg_pct_change = combined_pct.mean(axis=1)
    
    # Join with the date column
    time_series_df = time_series_df.set_index('日期')
    time_series_df['Cluster Pct Change'] = avg_pct_change
    time_series_df = time_series_df.reset_index()
    
    # Store the result
    result[label] = time_series_df[['日期', 'Cluster Pct Change']]


In [33]:
for lbl, df in result.items():
    print(lbl)
    display(df)

0


,日期,Cluster Pct Change
0,2024-01-02,0.098879
1,2024-01-03,0.089527
2,2024-01-04,-0.335709
3,2024-01-05,-0.836252
4,2024-01-08,-1.620333
...,...,...
238,2024-12-26,0.601142
239,2024-12-27,0.998875
240,2024-12-30,-0.553884
241,2024-12-31,-1.824351


1


,日期,Cluster Pct Change
0,2024-01-02,-0.409486
1,2024-01-03,-0.781873
2,2024-01-04,-0.667674
3,2024-01-05,-2.220514
4,2024-01-08,-1.893897
...,...,...
238,2024-12-26,1.406254
239,2024-12-27,0.060937
240,2024-12-30,-1.047039
241,2024-12-31,-2.891903


2


...

,日期,Cluster Pct Change
0,2024-01-02,0.529839
1,2024-01-03,-0.801127
2,2024-01-04,-0.303652
3,2024-01-05,-2.156569
4,2024-01-08,-2.098078
...,...,...
238,2024-12-26,1.694769
239,2024-12-27,0.602998
240,2024-12-30,-1.197817
241,2024-12-31,-2.637827


8


,日期,Cluster Pct Change
0,2024-01-02,2.255
1,2024-01-03,-1.015
2,2024-01-04,-0.440
3,2024-01-05,-2.570
4,2024-01-08,-1.155
...,...,...
238,2024-12-26,-0.805
239,2024-12-27,4.670
240,2024-12-30,-5.870
241,2024-12-31,-1.085


9


,日期,Cluster Pct Change
0,2024-01-02,-0.087778
1,2024-01-03,0.703333
2,2024-01-04,0.375556
3,2024-01-05,0.044444
4,2024-01-08,-0.344444
...,...,...
238,2024-12-26,-0.377778
239,2024-12-27,-0.178889
240,2024-12-30,0.666667
241,2024-12-31,-0.528889


## 将每一簇股票对于他们对应的指数进行capm分析得到每一alpha和beta

In [37]:
result_dfs = {}
for lbl, sub_dfs in dfs.items():
    result_dfs[lbl] = capm.CAPM(result[lbl], R_f = True, processed_dfs_dict = sub_dfs)

Reading files from: 'D:\Important Files\Repositories\Quantitative-Investment-Algorithms\Data\R_f.xlsx'
Reading files from: 'D:\Important Files\Repositories\Quantitative-Investment-Algorithms\Data\R_f.xlsx'
Reading files from: 'D:\Important Files\Repositories\Quantitative-Investment-Algorithms\Data\R_f.xlsx'
Reading files from: 'D:\Important Files\Repositories\Quantitative-Investment-Algorithms\Data\R_f.xlsx'
Reading files from: 'D:\Important Files\Repositories\Quantitative-Investment-Algorithms\Data\R_f.xlsx'
Reading files from: 'D:\Important Files\Repositories\Quantitative-Investment-Algorithms\Data\R_f.xlsx'
Reading files from: 'D:\Important Files\Repositories\Quantitative-Investment-Algorithms\Data\R_f.xlsx'
Reading files from: 'D:\Important Files\Repositories\Quantitative-Investment-Algorithms\Data\R_f.xlsx'
Reading files from: 'D:\Important Files\Repositories\Quantitative-Investment-Algorithms\Data\R_f.xlsx'
Reading files from: 'D:\Important Files\Repositories\Quantitative-Investm

In [42]:
for label, df in result_dfs.items():
    print(f"The label of this cluster is: {label}.")
    display(df)

The label of this cluster is: 0.


,Stock ID,alpha,beta
0,000001,0.089882,0.448474
1,000002,-0.093353,0.559599
2,000006,0.149951,0.913337
3,000031,-0.004469,1.340190
4,000032,-0.095022,1.265341
...,...,...,...
566,689009,0.210736,0.905842
567,832175,0.014342,1.409780
568,832522,0.457244,1.506850
569,835985,0.102711,1.430800


The label of this cluster is: 1.


,Stock ID,alpha,beta
0,000004,-0.036916,1.268916
1,000007,0.153928,0.614566
2,000020,-0.007214,0.905606
3,000503,-0.130488,1.053619
4,000504,-0.143216,1.078428
...,...,...,...
326,837592,0.471972,1.734251
327,872808,0.191106,1.699762
328,873122,0.366866,0.635054
329,873576,-0.090072,0.959095


The label of this cluster is: 2.


...

,Stock ID,alpha,beta
0,000028,0.006286,0.348755
1,000049,-0.046491,0.933749
2,000062,0.307116,0.707354
3,000066,0.150205,0.821162
4,000070,-0.107836,0.837692
...,...,...,...
989,872351,0.144309,1.001840
990,872541,0.275301,1.169002
991,872953,0.301217,1.249868
992,873152,0.200947,1.312922


The label of this cluster is: 8.


,Stock ID,alpha,beta
0,002513,-0.069588,1.463784
1,603557,0.069588,0.536216


The label of this cluster is: 9.


,Stock ID,alpha,beta
0,600028,-0.024602,0.858424
1,601288,0.035971,0.857114
2,601318,-0.003562,1.055365
3,601398,0.032726,0.779936
4,601658,-0.007892,1.253425
5,601668,-0.037458,1.300081
6,601857,-0.027201,1.223306
7,601939,0.013965,0.833540
8,601988,0.018053,0.838809


# 将得到的9个分组的数据保存下来至Output

In [44]:
for label, df in result_dfs.items():
    df.to_csv(os.path.join(gr.global_paths['Output'], f'{label}.csv'), index = False, encoding = 'utf-8')